# Homework 4 - Ensemble Methods and Decision Trees 
## CSCI 5622 - Spring 2019
***
**Name**: $Girish Narayanswamy$ 
***

This assignment is due on Canvas by **11.59 PM on Wednesday, March 20**. Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your classmates and instructors, but **you must write all code and solutions on your own**, and list any people or sources consulted.


## Dataset
***
Please do not change this class. We will use the MNIST dataset for this assignment. You have previously trained KNN, Logistic Regression on this dataset. You will now be using Ensemble methods and Decision Trees. This is a good opportunity to compare the results of different Machine Learning Algorithms on the dataset.

This is a binary classification task. We have only included 3's and 8's for this task.

In [1]:
import numpy as np
from sklearn.base import clone

from collections import Counter

In [2]:
class ThreesAndEights:
    """
    Class to store MNIST data
    """

    def __init__(self, location):

        import pickle, gzip

        # Load the dataset
        f = gzip.open(location, 'rb')

        # Split the data set
        train_set, valid_set, test_set = pickle.load(f)
    
        X_train, y_train = train_set
        X_valid, y_valid = valid_set

        # Extract only 3's and 8's for training set 
        self.X_train = X_train[np.logical_or( y_train==3, y_train == 8), :]
        self.y_train = y_train[np.logical_or( y_train==3, y_train == 8)]
        self.y_train = np.array([1 if y == 8 else -1 for y in self.y_train])
        
        # Shuffle the training data 
        shuff = np.arange(self.X_train.shape[0])
        np.random.shuffle(shuff)
        self.X_train = self.X_train[shuff,:]
        self.y_train = self.y_train[shuff]

        # Extract only 3's and 8's for validation set 
        self.X_valid = X_valid[np.logical_or( y_valid==3, y_valid == 8), :]
        self.y_valid = y_valid[np.logical_or( y_valid==3, y_valid == 8)]
        self.y_valid = np.array([1 if y == 8 else -1 for y in self.y_valid])
        
        f.close()

In [3]:
data = ThreesAndEights("data/mnist.pklz")

Feel free to explore this data and get comfortable with it before proceeding further.

## Bagging
Bootstrap aggregating, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

Given a standard training set $D$ of size n, bagging generates $N$ new training sets $D_i$, roughly each of size n * ratio, by sampling from $D$ uniformly and with replacement. By sampling with replacement, some observations may be repeated in each $D_i$ The $N$ models are fitted using the above $N$ bootstraped samples and combined by averaging the output (for regression) or voting (for classification). 

-Source [Wiki](https://en.wikipedia.org/wiki/Bootstrap_aggregating)

## Implementing Bagging [25 points]
***

We've given you a skeleton of the class `BaggingClassifier` below which will train a classifier based on the decision trees as implemented by sklearn. Your tasks are as follows, please approach step by step to understand the code flow:
* Implement `bootstrap` method which takes in two parameters (`X_train, y_train`) and returns a bootstrapped training set ($D_i$)
* Implement `fit` method which takes in two parameters (`X_train, y_train`) and trains `N` number of base models on different bootstrap samples. You should call `bootstrap` method to get bootstrapped training data for each of your base model
* Implement `voting` method which takes the predictions from learner trained on bootstrapped data points `y_hats` and returns final prediction as per majority rule. In case of ties, return either of the class randomly.
* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those. Please use the `voting` method to reach consensus on final prediction.

In [14]:
from sklearn.tree import DecisionTreeClassifier

class BaggingClassifier:
    def __init__(self, ratio = 0.20, N = 20, base=DecisionTreeClassifier(max_depth=4)):
        """
        Create a new BaggingClassifier
        
        Args:
            base (BaseEstimator, optional): Sklearn implementation of decision tree
            ratio: ratio of number of data points in subsampled data to the actual training data
            N: number of base estimator in the ensemble
        
        Attributes:
            base (estimator): Sklearn implementation of decision tree
            N: Number of decision trees
            learners: List of models trained on bootstrapped data sample
        """
        
        assert ratio <= 1.0, "Cannot have ratio greater than one"
        self.base = base
        self.ratio = ratio
        self.N = N
        self.learners = []
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone
        
        for i in range(self.N):
            h = clone(self.base) # clone the original model
            (Di, Di_y) = self.boostrap(X_train, y_train) # get the new bootstrapped dataset and classes
            h = h.fit(Di, Di_y) # fit to the new model 
            self.learners.append(h) # append model to array of models 
            
    def boostrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        
        n = X_train.shape[0] # size of total training data
        new_n = int(n*self.ratio) # get new size equal to old size*ratio
        idx = np.random.choice(n, new_n, replace=True)
        
        X_bootstrap = X_train[idx]
        y_bootstrap = y_train[idx]
        return (X_bootstrap, y_bootstrap)
    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        
        idx = 0
        y_hats = []
        
        for h in self.learners: # iterate through modesl
            y_hats.append(h.predict(X)) # append predictions
            
        y_hats = np.array(y_hats) # convert to np array
        y_hats_maj = self.voting(y_hats) # run voting 
        return y_hats_maj # find majority votes
    
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the 
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        
        # convert to classes -1 or 1
        y_hats_maj = y_hats.sum(axis=0) 
        y_hats_maj[y_hats_maj < 0] = -1
        y_hats_maj[y_hats_maj > 0] = 1
        y_hats_maj[y_hats_maj == 0] = np.random.choice([1,-1])
        return y_hats_maj
        
# Test for BeggingClassifier Function        
#clf = BaggingClassifier()
#clf.fit(data.X_train, data.y_train) 
#y_hats = clf.predict(data.X_valid)
#check = (data.y_valid == y_hats)
#check = np.array(check)
#print(check.sum(axis=0)/y_hats.shape[0])

## BaggingClassifier for Handwritten Digit Recognition [10 points]
***

After you've successfully completed `BaggingClassifier` find the optimal values of `ratio`, `N` and `depth` using k-fold cross validation. You are allowed to use sklearn library to split your training data in folds. Use the data from `ThreesAndEights` class initialized variable `data`. Hyperparameter tuning as you may have noticed is very important in Machine Learning.  

Justify why those values are optimal.

Report accuracy on the validation data using the optimal parameter values.

__Note__: This might take a little longer time than usual to run (i.e. several minutes). This is true for the ensemble methods you will implement below as well.

In [13]:
ratio_arr = [0.2, 0.4, 0.6, 0.8, 1.0]
N_arr = [10, 15, 20, 25, 30]
depth_arr = [2, 3, 4, 5, 6]

ratio_arr = np.array(ratio_arr)
N_arr = np.array(N_arr)
depth_arr = np.array(depth_arr)

max_acc = 0
max_ratio = 0
max_N = 0
max_depth = 0

for ratio_i in ratio_arr:
    for N_i in N_arr:
        for depth_i in depth_arr:
            
            clf = BaggingClassifier(ratio = ratio_i, N = N_i, base=DecisionTreeClassifier(max_depth = depth_i))
            clf.fit(data.X_train, data.y_train)
            y_hats = clf.predict(data.X_valid)
            
            check = (data.y_valid == y_hats)
            check = np.array(check)
            accuracy_i = check.sum(axis=0)/y_hats.shape[0]
            
            if(accuracy_i > max_acc):
                max_acc = accuracy_i
                max_ratio = ratio_i
                max_N = N_i
                max_depth = depth_i
                
print("Max Accuracy:", max_acc, "ratio:", max_ratio, "N:", max_N, "Depth:", max_depth)

Max Accuracy: 0.971554683668465 ratio: 1.0 N: 20 Depth: 6


<center>__Expected accuracy is about 97%__</center>

The program runs a triple nested for loop which iterates through 125 combinations of ratio, N, and depth values. The loop returns the maximum accuracy along with the ratio, N, and depth value which allow for the best accuracy. The accuracy is found to be 97.16% with a ratio = 1, N = 20, and depth = 6.

This however is not true for every run of the program. Because the seed is random the "optimal parameters" change every run of the program. An earlier run provided 97.35% with a ratio = 0.2, N = 15, and depth = 5.

A general trend is that the accuracy increases with higher values of ratio, and with medium depth (not lowest not highest). It is also worth noting that the execution time increases significantly with the ma

WARNING: This takes about 30 mins to run

# Random Decision Tree [35 points]

In this assignment you are going to implement a random decision tree using random vector method as discussed in the lecture.

Best split: One that achieves maximum reduction in gini index across multiple candidate splits. (decided by `candidate_splits` attribute of the class `RandomDecisionTree`)

Use `TreeNode` class as node abstraction to build the tree

You are allowed to add new attributes in the `TreeNode` and `RandomDecisionTree` class - if that helps.

Your tasks are as follows:
* Implement `gini_index` method which takes in class labels as parameter and returns the gini impurity as measure of uncertainty

* Implement `majority` method which picks the most frequent class label. In case of tie return any random class label

* Implement `find_best_split` method which finds the random vector/hyperplane which causes most reduction in the gini index. 

* Implement `build_tree` method which uses `find_best_split` method to get the best random split vector for current set of training points. This vector partitions the training points into two sets, and you should call `build_tree` method on two partitioned sets and build left subtree and right subtree. Use `TreeNode` as abstraction for a node.

> The method calls itself recursively to the generate left and right subtree till the point either `max_depth` is reached or no good random split is found.  When either of two cases is encountered, you should make that node as leaf and identify the label for that leaf to be the most frequent class (use `majority` method). Go through lecture slides for better understanding

* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those using the tree built. (`root` attribute of the class)

In [15]:
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.isLeaf = False
        self.label = None
        self.split_vector = None

    def getLabel(self):
        if not self.isLeaf:
            raise Exception("Should not to do getLabel on a non-leaf node")
        return self.label
    
class RandomDecisionTree:
            
    def __init__(self, candidate_splits = 100, depth = 10):
        """
        Args:
            candidate_splits (int) : number of random decision splits to test
            depth (int) : maximum depth of the random decision tree
        """
        self.candidate_splits = candidate_splits
        self.depth = depth
        self.root = None
    
    def fit(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        #print("Building Tree") 
        self.root = self.build_tree(X_train[:], y_train[:], 0)
        #print("Tree Built")
        return self
        
    def build_tree(self, X_train, y_train, height):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        
        node = TreeNode()
        # your logic here
        if len(X_train) <= 0 or len(y_train) < 0 or height < 0: # edge case recognition
            return
        
        # find split vector from best_split function
        split_vec, x_above, y_above, x_below, y_below = self.find_best_split(X_train, y_train)
        
        if height == self.depth: # reached max depth thus must be leaf
            node.label = self.majority(y_train)
            node.isLeaf = True # node is leaf
            node.split_vector = split_vec # set split to returned vector
            #print("leaf, max depth")
            return node
        
        elif self.gini_index(y_train) == 0: # leaf node as gini == 0
            node.label = self.majority(y_train)
            node.isLeaf = True # node is leaf
            node.split_vector = split_vec # set split to returned vector
            #print("leaf, gini 0")
            return node
        
        else: # if not leaf continue reccursion
            node.right = self.build_tree(x_above, y_above, height + 1) # recurse to right branch
            node.left = self.build_tree(x_below, y_below, height + 1) # recurse to left branch
            node.split_vector = split_vec # set split to returned vector
            #print("not leaf")
            return node
       
        return node
    
    def find_best_split(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
        """
    
        split_vector = None
        # your logic here
        class_above = None
        class_below = None
        
        gini_idx = self.gini_index(y_train) # calculate current gini index value
        gini_val = gini_idx
        info_gain = 0
        
        for i in range(self.candidate_splits):
            vec = np.random.normal(0,1,X_train.shape[1]) # random vector in n dim space   
            y_hats = np.matmul(X_train, vec) # dot values to find classes  
            
            # split into classes
            y_hats[y_hats < 0] = -1
            y_hats[y_hats > 0] = 1
            y_hats[y_hats == 0] = np.random.choice([1,-1])
            classes = (y_hats > 0) # class 1 above hyperplane
            
            # calculate gini above and below hyper plane
            gini_above = self.gini_index(y_train[classes])
            n_above = np.sum(classes)
            gini_below = self.gini_index(y_train[~classes])
            n_below = np.sum(~classes)
            
            # calculate information gain from gini index
            new_info_gain = gini_idx - gini_above*n_above/X_train.shape[0] - gini_below*n_below/X_train.shape[0]
            
            # save if better split than previous vector
            if new_info_gain > info_gain:
                info_gain = new_info_gain
                gini_val = min(gini_above, gini_below)
                split_vector = vec
                class_above = classes
                class_below = ~classes
        
        return split_vector, X_train[class_above, :], y_train[class_above], X_train[class_below, :], y_train[class_below]
        
    def gini_index(self, y):
        """
        Args:
            y (ndarray): [n_samples] ndarray of data
        """
        
        total = y.shape[0]
        (vals, counts) = np.unique(y, return_counts=True)
        sum_probs = 0
        
        for (val, count) in zip(vals, counts):
            prob = count / total
            sum_probs += prob**2
            
        gini_idx = 1 - sum_probs
        return gini_idx
    
    def majority(self, y):
        """
        Return the major class in ndarray y
        """
        
        y_maj = y.sum(axis=0)
        if y_maj < 0:
            maj = -1
        elif y_maj:
            maj = 1
        else:
            maj = np.random.choice([1,-1])
            
        return maj
    
    def predict(self, X):
        """
        BaggingClassifier prediction for new data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
            
        """
        
        y_hats = []
        for x in X: # choose instance in X
            node = self.root # start at root node
            
            while node.isLeaf != True: # continue while leaf is not reached
                
                if np.dot(x, node.split_vector) > 0.0: # dot to figure out which side of split_vec
                        node = node.right
                        
                elif np.dot(x, node.split_vector) < 0.0: # dot to figure out which side of split_vec
                        node = node.left
                        
                else: # if dot product returns 0 choose random classification
                    node = np.random.choice([node.left, node.right])
                    
            y_hats.append(node.label)
        
        return np.array(y_hats)
        
# Test Code
#clf = RandomDecisionTree()
#gini = clf.gini_index(data.y_train)
#maj = clf.majority(data.y_train)
#vec = clf.find_best_split(data.X_train, data.y_train)

## RandomDecisionTree for Handwritten Digit Recognition

After you've successfully completed `RandomDecisionTree`, and train using the default values in the constructor and report accuracy on the test set. Use the data from `ThreesAndEights` class initialized variable `data` 

In [12]:
# Test for RDT
rdt = RandomDecisionTree()
rdt.fit(data.X_train, data.y_train)
pred_val = rdt.predict(data.X_valid)

cnt = 0
for i, y in zip(pred_val, data.y_valid):
    if i == y:
        cnt = cnt + 1
        
scr = cnt / len(data.y_valid)
print("Accuracy:",scr)

Accuracy: 0.8911230995586071


<center>__Expected accuracy is about 90%__</center>

This simple test of the random decision tree returned an accuracy of 89.1 percent, very close to the expected 90% accuracy. 

# Random Forest [20 points]
Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set.

Random forest trains random decision trees on bootstrapped training points. Thus, you can implementation of methods (`bootstrap`, `predict`) from `BaggingClassifier` class directly. Only difference being, you have to use the `RandomDecisionTree` as base which you implemented previously instead of sklearn's implementation of `DecisionTreeClassifier`). Implement the `fit` method in the class below accordingly.

In [17]:
class RandomForest(BaggingClassifier):
    def __init__(self, ratio = 0.20, N = 10, max_depth = 10, candidate_splits = 100):
        self.ratio = ratio
        self.N = N
        self.learners = []
        self.candidate_splits = candidate_splits
        self.max_depth = max_depth
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone
        
        for i in range(self.N):
            (Di, Di_y) = self.boostrap(X_train, y_train) # get the new bootstrapped dataset and classes
            h = RandomDecisionTree() # fit to the new model 
            h = h.fit(Di, Di_y)
            self.learners.append(h) # append model to array of models 
            
    def boostrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        
        n = X_train.shape[0] # size of total training data
        new_n = int(n*self.ratio) # get new size equal to old size*ratio
        idx = np.random.choice(n, new_n, replace=True)
        
        X_bootstrap = X_train[idx]
        y_bootstrap = y_train[idx]
        return (X_bootstrap, y_bootstrap)
    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        
        idx = 0
        y_hats = []
        
        for h in self.learners:
            y_hats.append(h.predict(X))
            
        y_hats = np.array(y_hats)  
        y_hats_maj = self.voting(y_hats)
        return y_hats_maj
    
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the 
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        
        y_hats_maj = y_hats.sum(axis=0)
        y_hats_maj[y_hats_maj < 0] = -1
        y_hats_maj[y_hats_maj > 0] = 1
        y_hats_maj[y_hats_maj == 0] = np.random.choice([1,-1])
        return y_hats_maj

## RandomForest for Handwritten Digit Recognition [10 points]
***

After you've successfully completed `RandomForest` find the optimal values of `ratio`, `N`, `candidate_splits` and `depth` using k-fold cross validation on. Feel free to use sklearn library to split your training data. Use the data from `ThreesAndEights` class intialized variable `data`. 

Justify why those values are optimal.

Report best accuracy on the testing data using those optimal parameter values.

In [10]:
max_acc = 0
max_ratio = 0
max_N = 0
max_depth = 0
max_split = 0
ratio_arr = [0.2, 0.6]
N_arr = [10, 15, 20]
depth_arr = [5, 10]
canidate_splits_arr = [50, 100, 150]

for rat in ratio_arr:
    for Nval in N_arr:
        for depth in depth_arr:
            for split in canidate_splits_arr:
                
                
                # create forrest
                rf = RandomForest(ratio = rat, N = Nval, max_depth = depth, candidate_splits = split)
                rf.fit(data.X_train, data.y_train) # fit to model
                
                
                # predict classifications
                pred = rf.predict(data.X_valid) # get predicted values
                
                correct = (data.y_valid == pred) # number of correct classifications
                correct = np.array(correct) # convert to np array
                accuracy = ((correct.sum(axis=0))/pred.shape[0]) # calculate accuracy
                
                if accuracy > max_acc:
                    max_acc = accuracy
                    max_ratio = rat
                    max_N = Nval
                    max_depth = depth
                    max_split = split
                    print("ratio:", rat, "N:", Nval, "Depth:", depth, "Split:", split, "new max acc", max_acc)
                
# return best results
print("Max accuracy:", max_acc, ",Max ratio:", max_ratio, ",Max N:", max_N, ",Max depth:", max_depth, ",Max candidate split:", max_split)


ratio: 0.2 N: 10 Depth: 5 Split: 50 new max acc 0.9480137322216773
ratio: 0.2 N: 10 Depth: 5 Split: 100 new max acc 0.9499754781755763
ratio: 0.2 N: 10 Depth: 10 Split: 150 new max acc 0.9514467876410004
ratio: 0.2 N: 15 Depth: 5 Split: 50 new max acc 0.9602746444335458
ratio: 0.2 N: 15 Depth: 5 Split: 100 new max acc 0.9607650809220206
ratio: 0.2 N: 15 Depth: 10 Split: 100 new max acc 0.9641981363413438
ratio: 0.2 N: 20 Depth: 10 Split: 100 new max acc 0.9646885728298186
ratio: 0.6 N: 10 Depth: 5 Split: 50 new max acc 0.9661598822952427
ratio: 0.6 N: 15 Depth: 5 Split: 100 new max acc 0.9666503187837175
ratio: 0.6 N: 15 Depth: 10 Split: 50 new max acc 0.9725355566454145
Max accuracy: 0.9725355566454145 ,Max ratio: 0.6 ,Max N: 15 ,Max depth: 10 ,Max candidate split: 50


<center>__Expected accuracy is about 97%__</center>

The program runs a quadruple nested for loop which iterates through 36 combinations of ratio, N, depth values, and canidate split values. The loop returns the maximum accuracy along with the ratio, N, depth, and canidate splits value which allow for the best accuracy. The accuracy is found to be 97.25% with a ratio = 0.6, N = 15, and depth = 10, and canidate splits = 50.

This however is not true for every run of the program. As the splits and much of the code are random, the optimal parameters change every run of the program. 

WARNING: The above 36 combinations took 3 hours to run.